## Run the first three cells once after restarting the kernel

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
# Must set SNORKELDB before importing SnorkelSession
from set_env import set_env
set_env()
from snorkel import SnorkelSession
from snorkel.parser import TextDocPreprocessor
from snorkel.parser import CorpusParser
from snorkel.models import Document, Sentence
session = SnorkelSession()

In [2]:
print "Documents:", session.query(Document).count()
print "Sentences:", session.query(Sentence).count()

Documents: 400
Sentences: 95656


In [3]:
from snorkel.models import candidate_subclass


#GenePhenoPair = candidate_subclass('GenePhenoPair',['gene', 'pheno'])
PhenoPairComplex = candidate_subclass('ComplexPhenotypes',['descriptor', 'entity'])

## DO NOT RUN THIS CELL

In [6]:
#all_sents = set()
#docs = session.query(Document).order_by(Document.name).all()
#for i, doc in enumerate(docs):
    #for s in doc.sentences:
        #all_sents.add(s)
import cPickle
with open('small_data/pmcids_400.pkl', 'rb') as f:
    sent_dicts = cPickle.load(f)
train_ids, dev_ids, test_ids = set(sent_dicts['train']), set(sent_dicts['dev']), set(sent_dicts['test'])
all_ids = train_ids.union(dev_ids).union(test_ids)
# 40, 10, 10
train_sents, dev_sents, test_sents, all_sents = set(), set(), set(), set()
all_docs, train_docs, dev_docs, test_docs = set(), set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()

tair = []
with open('small_data/tair_labels.txt', 'rb') as f:
    for line in f:
        tair.append(line.strip())
tair = set(tair)

doc_sents = dict()
for doc_num, doc in enumerate(docs):
    #if len(all_docs) >=10: break
    #if len(train_docs) >= 40 and len(dev_docs) >= 10 and len(test_docs) >= 10:break
    doc_sents[doc_num] = set()
    for s in doc.sentences:
        doc_sents[doc_num].add(s)
        name = doc.name.split('-')[0]
        if name == 'PMC3503873':#in tair:
            all_docs.add(name)
            all_sents.add(s)
        if name in train_ids:
            train_docs.add(name)
            train_sents.add(s)
        elif name in dev_ids:
            dev_docs.add(name)
            dev_sents.add(s)
        elif name in test_ids:
            test_docs.add(name)
            test_sents.add(s)
        else:
            raise Exception('ID <{0}> not found in any id set'.format(doc.name))
    if name == 'PMC3503873': break

## RUN THIS CELL TO GET ALL DOCS LABELED BY TANYA 
### we also need to think about how to split for dev and test - not sure if we should do this here or somewhere else, if it is done here we need to update brat import to accomodate for dif splits because right now it only looks for split=0, but the good thing is the application of the labels is agnostic to the split so we basically just have to make sure we do the same process for each split

In [4]:
#all_sents = set()
#docs = session.query(Document).order_by(Document.name).all()
#for i, doc in enumerate(docs):
    #for s in doc.sentences:
        #all_sents.add(s)
import cPickle
with open('small_data/pmcids_400.pkl', 'rb') as f:
    sent_dicts = cPickle.load(f)
train_ids, dev_ids, test_ids = set(sent_dicts['train']), set(sent_dicts['dev']), set(sent_dicts['test'])
all_ids = train_ids.union(dev_ids).union(test_ids)
# 40, 10, 10
train_sents, dev_sents, test_sents, all_sents = set(), set(), set(), set()
all_docs, train_docs, dev_docs, test_docs = set(), set(), set(), set()
docs = session.query(Document).order_by(Document.name).all()

tair = []
with open('small_data/tair_labels.txt', 'rb') as f:
    for line in f:
        tair.append(line.strip())
tair = set(tair)

doc_splits = {}
for doc_num, doc in enumerate(docs):
    name = doc.name.split('-')[0]
    if name in tair:
        
        doc_splits[name] = set()
        for s in doc.sentences:
            all_sents.add(s)
            doc_splits[name].add(s)

In [5]:
print len(all_sents)
for key in doc_splits.keys():
    print key, len(doc_splits[key])
#print all_sents

18995
PMC2935865 108
PMC2935866 140
PMC2910046 126
PMC2905201 265
PMC3481204 130
PMC2444027 137
PMC3879159 358
PMC2803217 243
PMC2323576 488
PMC3878228 155
PMC3542051 135
PMC2785485 316
PMC4795774 326
PMC3178619 187
PMC2607017 284
PMC4283981 372
PMC3503873 274
PMC3913338 524
PMC4949643 236
PMC5156837 155
PMC3043248 168
PMC2803206 78
PMC3481211 180
PMC4340836 237
PMC2803208 104
PMC2664332 774
PMC4283595 203
PMC4339375 370
PMC3192994 228
PMC1557525 7
PMC2156172 285
PMC5384131 9
PMC3139611 130
PMC2890954 11
PMC1779593 7
PMC2877095 626
PMC2566869 154
PMC2920754 120
PMC2791118 170
PMC2736899 184
PMC2791111 197
PMC3504491 189
PMC2990762 878
PMC2478689 9
PMC2265440 215
PMC2908147 240
PMC4944173 175
PMC2551617 7
PMC2577656 8
PMC3935568 220
PMC3040660 356
PMC3017837 10
PMC2893204 357
PMC2803491 9
PMC2955748 370
PMC2295253 351
PMC2295252 160
PMC4955218 8
PMC3022413 175
PMC113751 6
PMC3913773 194
PMC3467300 607
PMC2999534 405
PMC3921138 232
PMC3878973 124
PMC5156734 120
PMC5132069 200
PMC3193000 

## Run the next four cells for Gene-Pheno Pair Extraction

### these can be skipped for phenotype

In [6]:
from candidate_extraction import PM, GM
from snorkel.candidates import Ngrams, CandidateExtractor


gene_ngrams = Ngrams(n_max=5)
pheno_ngrams = Ngrams(n_max=15)

cand_extractor = CandidateExtractor(GenePhenoPair, 
                                    [gene_ngrams, pheno_ngrams], [GM, PM],
                                    symmetric_relations=True, nested_relations=False)

pheno_cands_original.py:105: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return [pheno.lower() for pheno in terms if pheno.lower() not in blacklist and len(pheno)>1]
/home/CS341_Arabidopsis/venv/local/lib/python2.7/site-packages/nltk/stem/porter.py:274: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if word[-1] == 's':
extended_matchers.py:112: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  self.d = frozenset(self._stem(w) for w in list(self.d) if self._stem(w) not in self.blacklist and len(w)>2)


ImportError: cannot import name PM

In [95]:
print "Extracting Candidates..."

cand_extractor.apply(dev_sents, split=0)
dev_cands = session.query(GenePhenoPair).filter(GenePhenoPair.split==0).all()
print "Number of dev candidates:", len(dev_cands)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 559


In [96]:
from snorkel.viewer import SentenceNgramViewer
sv = SentenceNgramViewer(dev_cands, session, annotator_name = 'gold')

<IPython.core.display.Javascript object>

In [97]:
sv

## Run these next four cells for Pheno extraction for use w/ LFs

In [6]:
#from revised_extraction import PM
from pheno_candidates import PATO, OBO
from snorkel.candidates import Ngrams, CandidateExtractor

pheno_ngrams = Ngrams(n_max=15)

pheno_extractor_complex = CandidateExtractor(PhenoPairComplex, 
                                    [pheno_ngrams, pheno_ngrams], [PATO, OBO],
                                    nested_relations=True, self_relations=True)

pheno_candidates.py:105: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return [pheno.lower() for pheno in terms if pheno.lower() not in blacklist and len(pheno)>1]
/home/CS341_Arabidopsis/venv/local/lib/python2.7/site-packages/nltk/stem/porter.py:274: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if word[-1] == 's':
extended_matchers.py:112: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  self.d = frozenset(self._stem(w) for w in list(self.d) if self._stem(w) not in self.blacklist and len(w)>2)


In [8]:
print "Extracting Candidates..."
#clear dev and test splits
pheno_extractor_complex.clear(session, split=4)
pheno_extractor_complex.clear(session, split=5)

#extract all cands in a single split
pheno_extractor_complex.apply(all_sents, split=3)
pheno_cands_complex = session.query(PhenoPairComplex).filter(PhenoPairComplex.split==3).all()
print "Number of dev candidates:", len(pheno_cands_complex)

Extracting Candidates...
Clearing existing...
Running UDF...
[========================================] 100%%

Number of dev candidates: 53846


In [17]:
from snorkel.viewer import SentenceNgramViewer
svp = SentenceNgramViewer(pheno_cands_complex, session, annotator_name = 'gold_complex', height=400)

<IPython.core.display.Javascript object>

In [18]:
svp

## NO NEED TO RUN THESE LAST 3 CELLS FOR LF TESTS 
### they exist only for cand extraction unit testing/error analysis

In [25]:
print "Extracting Candidates..."

pheno_extractor.apply(doc_splits['PMC5181575'], split=0)
pheno_cands = session.query(PhenoPair).filter(PhenoPair.split==0).all()
print "Number of dev candidates:", len(pheno_cands)


Extracting Candidates...


KeyError: 'PMC5181575'

In [23]:
from snorkel.viewer import SentenceNgramViewer
svp = SentenceNgramViewer(pheno_cands, session, annotator_name = 'gold', height=400)

<IPython.core.display.Javascript object>

In [24]:
svp

In [10]:
pheno_extractor = CandidateExtractor(PhenoPair2, 
                                    [pheno_ngrams, pheno_ngrams], [PATO, OBO],
                                    nested_relations=True, self_relations=True)
pheno_extractor.clear(session, split=1)
pheno_extractor.clear(session, split=2)

In [14]:
print session.query(PhenoPair2).filter(PhenoPair2.split==2).all()

[]
